In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\littl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\littl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
#open the dataset 
df = pd.read_csv('./archive/spotify_songs.csv')
df['playlist_genre'].unique()

NameError: name 'pd' is not defined

In [3]:
#retrive the pop column 
pop = df[df['playlist_genre'] == 'pop']
pop = pop.sort_values(by='track_popularity', ascending=False)[['track_name','lyrics']]
# pop = df['lyrics']['track_popularity']
pop.head(15)
final = []
kevin = []
#add the stop words 
stop_words = set(stopwords.words('english'))
#add all the pop lyrics to a list 
for i in range(len(pop)):
    kevin.append(pop['lyrics'].iloc[i])
#add every lyric word by word to a list while removing the stop words 
for lyric in kevin:
    if isinstance(lyric, str):
        words = lyric.split()
        words = [word for word in words if word.lower() not in stop_words]
        final.extend(words)
Counter(final)

Counter({"I'm": 21009,
         'know': 11828,
         'like': 11663,
         'love': 10321,
         'got': 7469,
         'get': 6729,
         'wanna': 6564,
         'go': 6191,
         'want': 5948,
         'let': 5435,
         'never': 5291,
         "can't": 5235,
         'time': 5125,
         'feel': 4933,
         'me,': 4892,
         "I'll": 4879,
         'you,': 4835,
         'need': 4631,
         'see': 4475,
         'oh': 4406,
         'yeah': 4389,
         'make': 4372,
         'one': 4308,
         'say': 4132,
         'gonna': 4101,
         'way': 4054,
         'back': 3892,
         'Oh,': 3835,
         'come': 3631,
         "I've": 3616,
         "'Cause": 3595,
         'take': 3588,
         'could': 3368,
         'oh,': 3292,
         'night': 3197,
         'right': 3076,
         'give': 3038,
         'up,': 3037,
         'think': 3025,
         'it,': 2873,
         "ain't": 2823,
         'tell': 2781,
         'heart': 2676,
         'Ye

In [4]:
#repeat the same process as the previous cell but with rock songs
rock = df[df['playlist_genre'] == 'rock']
rock = rock.sort_values(by='track_popularity', ascending=False)[['track_name','lyrics']]
# pop = df['lyrics']['track_popularity']
finalRock = []
neil = []
stop_words = set(stopwords.words('english'))
for i in range(len(rock)):
    neil.append(rock['lyrics'].iloc[i])
    
for lyric in neil:
    if isinstance(lyric, str):
        words = lyric.split()
        words = [word for word in words if word.lower() not in stop_words]
        finalRock.extend(words)
Counter(finalRock)

Counter({"I'm": 8739,
         'love': 5250,
         'know': 4890,
         'like': 4086,
         'got': 4012,
         'get': 3607,
         "can't": 2967,
         'time': 2921,
         'never': 2831,
         'want': 2763,
         'go': 2741,
         'one': 2521,
         'away': 2517,
         'see': 2390,
         'take': 2376,
         'gonna': 2373,
         "I've": 2256,
         "I'll": 2252,
         'way': 2240,
         'come': 2216,
         'feel': 2180,
         'Oh,': 2070,
         'back': 2022,
         'make': 1924,
         'say': 1920,
         'night': 1899,
         'yeah': 1875,
         'wanna': 1847,
         'me,': 1744,
         'need': 1693,
         'la': 1690,
         'let': 1575,
         'right': 1524,
         'day': 1523,
         'tell': 1462,
         'life': 1456,
         'NA': 1445,
         'give': 1439,
         'could': 1435,
         'oh': 1434,
         'baby': 1426,
         'heart': 1394,
         'you,': 1336,
         'think': 1295

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# load the dataset again 
df = pd.read_csv('./archive/spotify_songs.csv')

# drop the rows that are missing lyrics 
df.dropna(subset=['lyrics'], inplace=True)


# split the dataset into lyrics and their associated genre 
X = df['lyrics']
y = df['playlist_genre']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# replace missing values with empty string 
X_train.fillna('', inplace=True)
X_test.fillna('', inplace=True)

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# train SVM classifier 
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

# predict the genre for the input lyrics 
y_pred = svm_classifier.predict(X_test_tfidf)

# evaluate the accuracy 
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5517999450398461


In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

dataset = pd.read_csv('./archive/spotify_songs.csv')

dataset['lyrics'].fillna('', inplace=True)

# convert lyrics to tokens 
tfidf_vectorizer = TfidfVectorizer(max_features=3000) 

# fit the vectorizer to the entire dataset 
X = tfidf_vectorizer.fit_transform(dataset['lyrics'])
y = dataset['playlist_genre']

# train the SVM classifier 
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X, y)

# Function to predict the genre of a song based on its lyrics
def predict_genre(lyrics):
    # Transform the input lyrics using the trained TF-IDF vectorizer
    lyrics_features = tfidf_vectorizer.transform([lyrics])
    # Predict the genre using the trained SVM classifier
    prediction = svm_classifier.predict(lyrics_features)
    return prediction[0]

# user input lyrics to test prediction 
user_lyrics = "You said you hated the ocean, but you're surfin now I said I'd love you for life, but I just sold our house. We were kids at the start, I guess we're grown-ups now, mm. Couldn't ever imagine even havin' doubts. But not everything works out, no"

# Predict the genre
predicted_genre = predict_genre(user_lyrics)
print("Predicted Genre:", predicted_genre)

Predicted Genre: pop


In [42]:
dfFein = pd.read_csv('./testsetGenre.csv')


In [43]:

dfFein.dropna(inplace=True)
lyricsList = dfFein['lyrics'].tolist()
titleList = dfFein['title '].tolist()
j = 0
for i in range(len(lyricsList)):
#     print(lyricsList[i])
    user_lyrics = lyricsList[i]
    predicted_genre = predict_genre(user_lyrics)
    print(f"Predcited Genre of {titleList[i]}: {predicted_genre}")
    print('---------------------------------------')
#for i in lyricsList:

#    user_lyrics = i
 #   predicted_genre = predict_genre(user_lyrics)
  #  print(f"Predicted Genre of {dfFein['title '].iloc(j)}: ", predicted_genre)
   # print("\n")
    #j+=1
    

Predcited Genre of Bam Bam: pop
---------------------------------------
Predcited Genre of Despacito: latin
---------------------------------------
Predcited Genre of Friendly Fire: edm
---------------------------------------
Predcited Genre of Enemy: rock
---------------------------------------
Predcited Genre of Texas Hold Em': r&b
---------------------------------------
Predcited Genre of Udhungada Sangu: pop
---------------------------------------
Predcited Genre of Gully: rap
---------------------------------------
Predcited Genre of Leo Das: pop
---------------------------------------
Predcited Genre of unravel: rock
---------------------------------------
Predcited Genre of Tejas: r&b
---------------------------------------


In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load dataset containing song lyrics and corresponding genres
df = pd.read_csv('./archive/spotify_songs.csv')

# drop rows with missing lyrics 
df.dropna(subset=['lyrics'], inplace=True)

# remove stop words
stop_words = set(stopwords.words('english'))
df['lyrics'] = df['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Split dataset into features (lyrics) and target (genre)
X = df['lyrics']
y = df['playlist_genre']

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)

#replace nan values with empty strings to not stop processing 
X_train.fillna('', inplace=True)
X_test.fillna('', inplace=True)

# Feature extraction using TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_tfidf, y_train)

# Predict genre labels for test data
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5302197802197802


In [45]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

#load the dataset 
dataset = pd.read_csv('./archive/spotify_songs.csv')

dataset['lyrics'].fillna('', inplace=True)

# convert lyrics to numerical features 
tfidf_vectorizer = TfidfVectorizer(max_features=3000)  

# Fit the TF-IDF vectorizer on the entire dataset
X = tfidf_vectorizer.fit_transform(dataset['lyrics'])
y = dataset['playlist_genre']

# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X, y)

# Function to predict the genre of a song based on its lyrics
def predict_genre(lyrics):
    # Transform the input lyrics using the trained TF-IDF vectorizer
    lyrics_features = tfidf_vectorizer.transform([lyrics])
    # Predict the genre using the trained SVM classifier
    prediction = svm_classifier.predict(lyrics_features)
    return prediction[0]

# User input
user_lyrics = "You said you hated the ocean, but you're surfin now I said I'd love you for life, but I just sold our house. We were kids at the start, I guess we're grown-ups now, mm. Couldn't ever imagine even havin' doubts. But not everything works out, no"

# Predict the genre
predicted_genre = predict_genre(user_lyrics)
print("Predicted Genre:", predicted_genre)
dfFein.dropna(inplace=True)
lyricsList = dfFein['lyrics'].tolist()
titleList = dfFein['title '].tolist()
j = 0
for i in range(len(lyricsList)):
#     print(lyricsList[i])
    user_lyrics = lyricsList[i]
    predicted_genre = predict_genre(user_lyrics)
    print(f"Predcited Genre of {titleList[i]}: {predicted_genre}")
    print('---------------------------------------')
#for i in lyricsList:

#    user_lyrics = i
 #   predicted_genre = predict_genre(user_lyrics)
  #  print(f"Predicted Genre of {dfFein['title '].iloc(j)}: ", predicted_genre)
   # print("\n")
    #j+=1
    

Predicted Genre: pop
Predcited Genre of Bam Bam: pop
---------------------------------------
Predcited Genre of Despacito: latin
---------------------------------------
Predcited Genre of Friendly Fire: edm
---------------------------------------
Predcited Genre of Enemy: rock
---------------------------------------
Predcited Genre of Texas Hold Em': r&b
---------------------------------------
Predcited Genre of Udhungada Sangu: pop
---------------------------------------
Predcited Genre of Gully: rap
---------------------------------------
Predcited Genre of Leo Das: pop
---------------------------------------
Predcited Genre of unravel: rock
---------------------------------------
Predcited Genre of Tejas: r&b
---------------------------------------
